# 1. Packages import

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

# 2. Data retrieval

In [2]:
def clean_sheet(dataframe):
    '''
    cleaning function for raw CSV import
    '''
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [3]:
def price_date_transform(CSV_date,index=False):
    '''
    Transform the CSV price style string into dateframe string style
    The CSV date follows US style which is MM/DD/YYYY
    '''
    if index==False:
        timestamp=pd.Timestamp(int(CSV_date[CSV_date.find("/",3)+1:]),
                            int(CSV_date[:CSV_date.find("/")]),
                            int(CSV_date[CSV_date.find("/",1)+1:CSV_date.find("/",3)]))
        return timestamp.strftime("%d/%b/%Y")
    else:
        timestamp=pd.Timestamp(int(CSV_date[-4:]),int(CSV_date[3:5]),int(CSV_date[:2]))
        return timestamp

In [4]:
def fundamental_date_transform(CSV_date):
    '''
    Transform the fundamental style string into dateframe string style
    CSV date follow following style yyyy-mm-dd or MM/DD/YYYY
    '''
    if '-' in CSV_date:
        timestamp=pd.Timestamp(int(CSV_date[:4]),
                            int(CSV_date[5:7]),
                            int(CSV_date[8:]))
    else:
        timestamp=pd.Timestamp(int(CSV_date[CSV_date.find("/",3)+1:]),
                               int(CSV_date[CSV_date.find("/",1)+1:CSV_date.find("/",3)]),
                               int(CSV_date[:CSV_date.find("/")]))        
    return timestamp

In [5]:
def CSV_price(region,start,end,VWAP):
    '''
    Grab the pricing data from CSV
    region= US, Europe,Asia,or Canada
    start,end are in year
    key is the reference to search
    return the target price dataframe with timestamp on the column
    '''
    mylist=[]
    for year in range(start,end+1):
        if VWAP==False:
            csv=pd.read_csv(r"C:\Users\Eric.Li\Documents\Post result data\{0} CSV\{0}_price_{1}.csv".format(region,year))
        else:
            csv=pd.read_csv(r"C:\Users\Eric.Li\Documents\Post result data\{0} CSV\{0}_VWAP_{1}.csv".format(region,year))
        data=csv.set_index("Ticker")
        adj_data=data.loc[[x for x in data.index if type(x)==str]].replace('#N/A N/A','').replace(' #N/A N/A ','')
        adj_data=adj_data.loc[[x for x in adj_data.index if len(x)>0]]
        mylist.append(adj_data)

    price=pd.concat(mylist,axis=1)
    price=price.apply(lambda x:pd.to_numeric(x),axis=1)
    
    csv_index=pd.read_csv(r"C:\Users\Eric.Li\Documents\Post result data\{0} CSV\{0}_price_index.csv".format(region))
    data_index=csv_index.set_index("Ticker").T
    price_index=data_index.replace('#N/A N/A','')
    #price_index=data_index.apply(lambda x:pd.to_numeric(x),axis=1)
    
    price.columns=[price_date_transform(i) for i in price.columns]
    '''
    Need to sort the columns for index price, and then transform to date string
    '''
    price_index.columns=[price_date_transform(i) for i in price_index.columns]
    #price_index=price_index.reindex(sorted(price_index.columns))
    #price_index.columns=[i.strftime("%d/%b/%Y") for i  in price_index.columns]
    
    abs_return=price.diff(1,axis=1)/price.shift(1,axis=1)
    abs_return_index=price_index.diff(1,axis=1)/price_index.shift(1,axis=1)
    return price,abs_return,price_index,abs_return_index

In [6]:
def CSV_EPS(region,start,end):
    '''
    Grab the pricing data from CSV database
    region= US, Europe,Asia,or Canada
    start,end are in year
    key is the reference to search
    return the target EPS dataframe with timestamp on the column
    '''
    mylist=[]
    for year in range(start,end+1):
        csv=pd.read_csv(r"C:\Users\Eric.Li\Documents\Post result data\{0} CSV\{0}_EPS_{1}.csv".format(region,year))
        data=csv.set_index("Ticker")
        adj_data=data.loc[[x for x in data.index if type(x)==str]].replace('#N/A N/A','').replace(" #N/A N/A ","")
        adj_data=adj_data.loc[[x for x in adj_data.index if len(x)>0]]
        mylist.append(adj_data)

    EPS=pd.concat(mylist,axis=1)
    EPS=EPS.apply(lambda x:pd.to_numeric(x),axis=1)
    
    EPS.columns=[price_date_transform(i) for i in EPS.columns]
    return EPS

In [7]:
def revision_calc(ticker,date,EPS_df,period):
    '''
    Calculate revision from ticker and reference date
    '''
    if type(date)==pd.lib.tslib.NaTType:
        return None
    elif type(date)==pd.lib.tslib.Timestamp:
        date=date.strftime("%d/%b/%Y")
    elif type(date)==str:
        date=date
    
    if ticker in EPS_df.index:
        eps_series=EPS_df.loc[ticker]
        date_series=eps_series.index.tolist()
        if date in date_series:
            day0=date_series.index(date)
            post_series=eps_series.iloc[day0+period-1:day0+period+10]
            pre_series=eps_series.iloc[day0-10:day0]
            if len(post_series.dropna())==0 or len(pre_series.dropna())==0 or pre_series.dropna().iloc[-1]==0:
                revision=None
            else:
                try:
                    revision=np.divide(post_series.dropna().iloc[0],pre_series.dropna().iloc[-1])-1
                except:
                    revision=None
            return revision
        else:
            return None
    else:
        return None

In [53]:
csv=pd.read_csv(r'C:\Users\Eric.Li\Documents\Post result code\{0}.csv'.format('US'))

In [78]:
csv["SI"].dropna().sort_values().iloc[40]

'#N/A Requesting Data...'

In [80]:
def CSV_fundamentals(region,price,EPS_df,revision_period,min_history,min_vol,use_cache):
    '''
    Grab the fundamental data from the spreadsheet
    region= US, Europe,Asia,or Canada
    return the post result fundamental dataframe
    '''
    
    if use_cache is True:
        try:
            data=pd.read_csv(r'C:\Users\Eric.Li\Documents\Post result code\{0}{1}_clean_reg.csv'.format(region,\
                                                                                                        str(revision_period)))
            new_index=pd.MultiIndex.from_tuples(list(zip(data.iloc[:,0],data.iloc[:,1],data.iloc[:,2],data.iloc[:,3])))
            data.index=new_index
            target_data=data.iloc[:,4:]
        except:
            print("No such file!")
    else:
        csv=pd.read_csv(r'C:\Users\Eric.Li\Documents\Post result code\{0}.csv'.format(region))
        data=csv.set_index("Ticker").drop_duplicates().replace('#N/A Invalid Security','').\
        replace('#N/A Requesting Data...','')

        data=data[data.index!='']
        data=data.dropna(how="all")

        data["date_copy"]=[fundamental_date_transform(i) for i in data["Date"].copy()]
        data["ticker_copy"]=data.index
        data=data.copy().sort_values(by=["ticker_copy","date_copy"])
        data["next_date"]=data["date_copy"].shift(-1)
        data["ticker_copy"]=data["ticker_copy"].shift(-1)
        data["Date"]=data["date_copy"].copy().apply(lambda x: x.strftime("%d/%b/%Y") if x!='' else np.nan)
        data["Next"]=data.apply(lambda x: x["next_date"].strftime("%d/%b/%Y") if type(x["next_date"])==pd.Timestamp and \
                                x.name==x["ticker_copy"] else np.nan,axis=1)

#         data["period"]=data.apply(lambda x:str(pd.Timestamp(datetime.strptime(x["Date"],"%d/%b/%Y")).year)\
#                                             +" "+str(pd.Timestamp(datetime.strptime(x["Date"],"%d/%b/%Y")).quarter),\
#                                             axis=1)

        data["end_period"]=data.apply(lambda x: pd.offsets.BQuarterEnd().rollforward(x["date_copy"]).strftime("%d/%b/%Y"),\
                                      axis=1)

        data.index=pd.MultiIndex.from_tuples(list(zip(data.index,data["Date"],data["Next"],data["end_period"])))

        del data["ticker_copy"]
        del data["date_copy"]
        del data["next_date"]
        del data["Revision"]
        del data["end_period"]
        del data["Next"]

        for s in ["Market cap","Volume","SI","Broker"]:
            try:
                data[s]=pd.to_numeric(data[s])
            except KeyError:
                pass

        '''
        Add more realistic version of revision
        '''
        data["Revision_real"]=data.apply(lambda x: revision_calc(x.name[0],x.name[1],EPS_df,revision_period),axis=1)
        
        data["Revision_20"]=data.apply(lambda x: revision_calc(x.name[0],x.name[1],EPS_df,20),axis=1)
        data=data[(data["Revision_20"]>0)|(data["Revision_20"]<0)]

        '''
        take out data with zero or none revision/market cap
        '''
        data=data[(data["Revision_real"]>0)|(data["Revision_real"]<0)]
        data=data[(data["Market cap"]>500)] #universe above 500mn
        
        '''
        take out cases where there is a short history
        '''
        count_history=data.apply(lambda x: price.loc[x.name[0],:x.name[1]][-2*min_history:].count() if x.name[1] in \
                                   price.columns else None,axis=1)
        
        data=data.copy()[count_history>min_history]
        
        '''
        Add momentum
        '''
        data["mom"]=data.apply(lambda x: price.loc[x.name[0],:x.name[1]][-260:-23].dropna()[-1]/\
                               price.loc[x.name[0],:x.name[1]][-260:-23].dropna()[0] if x.name[1] in \
                               price.columns else None, axis=1)
        
        data=data[(data["mom"]>0)|(data["mom"]<0)]
        
        data["mom_short"]=data.apply(lambda x: price.loc[x.name[0],:x.name[1]][-24:-1].dropna()[-1]/\
                               price.loc[x.name[0],:x.name[1]][-24:-1].dropna()[0] if x.name[1] in \
                               price.columns else None, axis=1)      

        '''
        Add historic volatility
        '''
        
        abs_return=price.diff(1,axis=1)/price.shift(1,axis=1)
        
        data["30d_vol"]=data.apply(lambda x: abs_return.loc[x.name[0],:x.name[1]][-31:-1].std() if x.name[1] in \
                                   abs_return.columns else None,axis=1)      
        
        data=data[data["30d_vol"]>=min_vol]
        
        '''
        Final cleaning and export the data
        '''
        target_data=data.drop_duplicates()
        target_data.to_csv(r'C:\Users\Eric.Li\Documents\Post result code\{0}{1}_clean_reg.csv'.format(region,\
                                                                                                      str(revision_period)))
    return target_data

## 2.1 - US data

In [9]:
US_price,abs_return_US,US_index_price,abs_return_index_US=CSV_price('US',2006,2019,False)

In [10]:
US_VWAP,abs_return_VWAP_US,US_index_VWAP,abs_return_VWAP_index_US=CSV_price('US',2006,2019,True)

In [11]:
US_EPS =CSV_EPS("US",2006,2019)

In [81]:
US1=CSV_fundamentals("US",US_price,US_EPS,1,120,0.005,False)

## 2.2 - Asian data

In [ ]:
# Asia index mmapping
Asia_mapping=clean_sheet(pd.read_csv("Asia mapping.csv")).dropna(how="all").iloc[:,:1]

In [ ]:
Asia_price,abs_return_Asia,Asia_index_price,abs_return_index_Asia=CSV_price('Asia',2010,2019,False)

In [ ]:
Asia_EPS=CSV_EPS("Asia",2010,2019)

In [ ]:
Asia2_old=CSV_fundamentals("Asia",abs_return_Asia,Asia_EPS,2,"old")

In [ ]:
Asia2_new=CSV_fundamentals("Asia",abs_return_Asia,Asia_EPS,2,"new")

## 2.3 - European data

In [ ]:
Europe_price,abs_return_Europe,Europe_index_price,abs_return_index_Europe=CSV_price('Europe',2010,2019,False)

In [ ]:
Europe_VWAP,abs_return_VWAP_Europe,Europe_index_VWAP,abs_return_index_Europe=CSV_price('Europe',2010,2019,True)

In [ ]:
Europe_EPS=CSV_EPS("Europe",2010,2019)

In [ ]:
Europe2_old=CSV_fundamentals("Europe",abs_return_Europe,Europe_EPS,2,'old')

In [ ]:
Europe2_new=CSV_fundamentals("Europe",abs_return_Europe,Europe_EPS,2,'new')

# 3. Utility functions

In [212]:
def return_calc(ticker,date,price_df,index_df,period_tuple,abs_rel):
    '''
    Calculate return from the period tuple, abs_return if assigned abs
    Day starts from zero, so 1 means start the return calculation one day after result, second element is the number of days
    '''
    if type(date)==pd.tslib.NaTType:
        return None
    elif type(date)==pd.tslib.Timestamp:
        date=date.strftime("%d/%b/%Y")
    elif type(date)==str:
        date=date
    price_series=price_df.loc[ticker].dropna()
    date_series=price_series.index.tolist()
    
    index_data_series=index_df.index.tolist()
    
    if date in date_series:
        day0=date_series.index(date)
        day0_index=index_data_series.index(date)
        
        start_price=price_series.iloc[:day0+period_tuple[0]].dropna().iloc[-1]
        end_price=price_series.iloc[:day0+period_tuple[0]+period_tuple[1]].dropna().iloc[-1]
        
        target_series=price_series.iloc[day0+period_tuple[0]-2:day0+period_tuple[0]+period_tuple[1]]
        
        if start_price!=0:
            abs_return=end_price/start_price-1
        else:
            abs_return=None
        

        
        if abs_rel=='abs':
            target_return=abs_return
        else:

            start_index=index_df.iloc[:day0_index+period_tuple[0]].dropna().iloc[-1]
            end_index=index_df.iloc[:day0_index+period_tuple[0]+period_tuple[1]].dropna().iloc[-1]
            if start_index!=0:
                index_return=end_index/start_index-1
            else:
                index_return=None

            if abs_return is None or index_return is None:
                target_return=None
            else:
                target_return=abs_return-index_return
        return target_return
    else:
        return None

In [90]:
target_US=US1.copy()

In [213]:
target_US["pre_announcement_return"]=np.log(1+target_US.apply(lambda x:return_calc(x.name[0],x.name[1],US_price,\
                                                                           US_index_price.loc['SPX Index'],(-11,10),'rel')\
                                                              ,axis=1))

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [214]:
target_US["EAR"]=np.log(1+target_US.apply(lambda x:return_calc(x.name[0],x.name[1],US_price,\
                                                                           US_index_price.loc['SPX Index'],(0,2),'rel'),axis=1))

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [244]:
target_US["target_return"]=target_US.apply(lambda x:return_calc(x.name[0],x.name[1],US_price,\
                                                                           US_index_price.loc['SPX Index'],(2,20),'rel'),axis=1)

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
for i in target_US["Supersector"].dropna().unique():
    target_US[i]=(target_US["Supersector"]==i)*1.0

In [179]:
target_US["Revision_20"][target_US["Revision_20"]>0.99]=0.99
target_US["Revision_20"][target_US["Revision_20"]<-0.99]=-0.99

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

In [191]:
target_US["mom"]=np.log(target_US["mom"])

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [307]:
target_US["target_return_positive"]=(target_US["target_return"]>0.02)*1.0

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [308]:
target_US["target_return_negative"]=(target_US["target_return"]<-0.02)*1.0

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [245]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [379]:
x=target_US[['Market cap','Revision_real','mom','30d_vol','pre_announcement_return','EAR','Broker','SI']+target_US["Supersector"].dropna().unique().tolist()]
y=target_US['target_return']
y_log_positive=target_US['target_return_positive']
y_log_negative=target_US['target_return_negative']

## Linear regression

In [370]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [359]:
regr = LinearRegression()

In [360]:
regr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [361]:
x_result=x_test.copy()

In [362]:
x_result["y_predict"]=regr.predict(x_test)

In [363]:
y_result=y_test.to_frame()

In [364]:
regr.predict(x_test)

array([-0.00956222,  0.00357225,  0.00885337, ...,  0.0111883 ,
        0.008259  , -0.01010672])

In [365]:
y_result["predict"]=regr.predict(x_test)

In [371]:
import statsmodels.api as sm

In [372]:
model=sm.OLS(y_train,x_train).fit()

In [373]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          target_return   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     12.66
Date:                Tue, 18 Jun 2019   Prob (F-statistic):           3.53e-46
Time:                        16:30:01   Log-Likelihood:                 58431.
No. Observations:               55771   AIC:                        -1.168e+05
Df Residuals:                   55748   BIC:                        -1.166e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Market cap                   -2.02e-08   9.76e-09     -2.069      0.039   -3.93e-08   -1.06e-09
Revision_real                   0.0144      0.003      5.339      0.000       0.009       0.020
mom                            -0.0052      0.001     -5.266      0.000      -0.007      -0.003
30d_vol                        -0.1363      0.027     -4.990      0.000      -0.190      -0.083
Utilities                       0.0044      0.002      2.065      0.039       0.000       0.008
Oil & Gas                      -0.0047      0.002     -3.082      0.002      -0.008      -0.002
Health Care                     0.0149      0.001     11.835      0.000       0.012       0.017
Personal & Household Goods      0.0053      0.002      3.163      0.002       0.002       0.009
Retail                          0.0073      0.001      5.085      0.000       0.005       0.010
Technology                      0.0125      0.001     10.499      0.000       0.010       0.015
Telecommunications              0.0109      0.003      3.155      0.002       0.004       0.018
Financial Services              0.0018      0.002      0.976      0.329      -0.002       0.005
Travel & Leisure                0.0038      0.002      2.126      0.034       0.000       0.007
Industrial Goods & Services     0.0066      0.001      6.339      0.000       0.005       0.009
Banks                           0.0051      0.002      3.105      0.002       0.002       0.008
Media                           0.0041      0.002      1.887      0.059      -0.000       0.008
Automobiles & Parts             0.0002      0.003      0.077      0.938      -0.006       0.006
Insurance                       0.0073      0.002      3.794      0.000       0.004       0.011
Basic Resources                 0.0020      0.003      0.778      0.437      -0.003       0.007
Real Estate                    -0.0023      0.006     -0.383      0.702      -0.014       0.009
Food & Beverage                 0.0143      0.002      6.481      0.000       0.010       0.019
Construction & Materials        0.0103      0.002      4.436      0.000       0.006       0.015
Chemicals                       0.0079      0.002      3.234      0.001       0.003       0.013
==============================================================================
Omnibus:                    17115.944   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           471793.137
Skew:                           0.889   Prob(JB):                         0.00
Kurtosis:                      17.137   Cond. No.                     3.01e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The co

## logistic regression

In [375]:
y_log_positive

0111145D US  23/Feb/2007  27/Apr/2007  30/Mar/2007    1.0
             01/Nov/2007  22/Feb/2008  31/Dec/2007    1.0
             22/Feb/2008  01/May/2008  31/Mar/2008    0.0
             01/May/2008  04/Aug/2008  30/Jun/2008    1.0
             03/Nov/2008  25/Feb/2009  31/Dec/2008    0.0
             01/May/2009  31/Jul/2009  30/Jun/2009    0.0
             30/Oct/2009  24/Feb/2010  31/Dec/2009    0.0
             24/Feb/2010  03/May/2010  31/Mar/2010    0.0
             03/May/2010  03/Aug/2010  30/Jun/2010    1.0
             03/Aug/2010  02/Nov/2010  30/Sep/2010    0.0
             01/Nov/2011  NaN          30/Dec/2011    1.0
0167866D US  06/May/2008  06/Aug/2008  30/Jun/2008    1.0
             06/Aug/2008  06/Nov/2008  30/Sep/2008    0.0
             06/Nov/2008  25/Feb/2009  31/Dec/2008    0.0
             25/Feb/2009  06/May/2009  31/Mar/2009    1.0
             06/May/2009  05/Aug/2009  30/Jun/2009    0.0
             05/Aug/2009  05/Nov/2009  30/Sep/2009    0.0
             0

In [313]:
from sklearn.linear_model import LogisticRegression

In [314]:
x_train, x_test, y_train_positive, y_test_positive = train_test_split(x, y_log_positive, test_size=0.2, random_state=0)

In [315]:
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(x_train, y_train_positive)

In [316]:
clf.score(x_train, y_train_positive)

0.6245898405981604

In [327]:
test_result_log=x_test.copy()

In [328]:
test_result_log["predict"]=clf.predict(x_test)

In [330]:
test_result_log["predict"].sum()

0.0

In [320]:
model_log=sm.Logit(y_train_positive,x_train).fit()

Optimization terminated successfully.
         Current function value: 0.659017
         Iterations 5


In [321]:
model_log.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Results: Logit
============================================================================
Model:                 Logit                   No. Iterations:    5.0000    
Dependent Variable:    target_return_positive  Pseudo R-squared:  0.004     
Date:                  2019-06-18 16:19        AIC:               73562.1187
No. Observations:      55771                   BIC:               73803.2020
Df Model:              26                      Log-Likelihood:    -36754.   
Df Residuals:          55744                   LL-Null:           -36908.   
Converged:             1.0000                  Scale:             1.0000    
----------------------------------------------------------------------------
                             Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------------
Market cap                  -0.0000   0.0000  -5.2682 0.0000 -0.0000 -0.0000
Revision_real                0.1125   0.0656   1.7143 0.0865 -0.0161  0.2411
mom                         -0.0864   0.0244  -3.5432 0.0004 -0.1343 -0.0386
30d_vol                      5.1430   0.6812   7.5496 0.0000  3.8078  6.4781
pre_announcement_return     -0.1745   0.1369  -1.2742 0.2026 -0.4429  0.0939
EAR                         -0.0508   0.1072  -0.4737 0.6357 -0.2608  0.1593
Broker                      -0.0018   0.0014  -1.2600 0.2077 -0.0046  0.0010
SI                           0.0033   0.0017   1.9528 0.0508 -0.0000  0.0066
Utilities                   -0.6869   0.0559 -12.2908 0.0000 -0.7965 -0.5774
Oil & Gas                   -0.7155   0.0482 -14.8573 0.0000 -0.8099 -0.6211
Health Care                 -0.4627   0.0396 -11.6751 0.0000 -0.5404 -0.3850
Personal & Household Goods  -0.6151   0.0470 -13.0782 0.0000 -0.7072 -0.5229
Retail                      -0.5829   0.0447 -13.0457 0.0000 -0.6705 -0.4954
Technology                  -0.4604   0.0378 -12.1648 0.0000 -0.5346 -0.3862
Telecommunications          -0.5054   0.0877  -5.7621 0.0000 -0.6774 -0.3335
Financial Services          -0.7107   0.0512 -13.8861 0.0000 -0.8111 -0.6104
Travel & Leisure            -0.6037   0.0507 -11.9134 0.0000 -0.7030 -0.5044
Industrial Goods & Services -0.6189   0.0341 -18.1624 0.0000 -0.6857 -0.5521
Banks                       -0.7334   0.0488 -15.0251 0.0000 -0.8291 -0.6377
Media                       -0.6649   0.0591 -11.2443 0.0000 -0.7808 -0.5490
Automobiles & Parts         -0.7313   0.0783  -9.3407 0.0000 -0.8848 -0.5779
Insurance                   -0.7465   0.0522 -14.3095 0.0000 -0.8487 -0.6443
Basic Resources             -0.6660   0.0654 -10.1861 0.0000 -0.7941 -0.5378
Real Estate                 -0.7311   0.1474  -4.9616 0.0000 -1.0200 -0.4423
Food & Beverage             -0.4606   0.0575  -8.0034 0.0000 -0.5734 -0.3478
Construction & Materials    -0.4590   0.0602  -7.6240 0.0000 -0.5771 -0.3410
Chemicals                   -0.5775   0.0627  -9.2110 0.0000 -0.7004 -0.4547
============================================================================

"""

In [387]:
def EAR_calc(ticker,date,return_df,EAR_period,vol_lookback):
    '''
    Calculate EAR from ticker and reference date
    '''
    if type(date)==pd.tslib.NaTType:
        return None
    elif type(date)==pd.tslib.Timestamp:
        date=date.strftime("%d/%b/%Y")
    elif type(date)==str:
        date=date
    return_series=return_df.loc[ticker].dropna()
    date_series=return_series.index.tolist()
    if date in date_series:
        day0=date_series.index(date)
        post_series=return_series.iloc[day0:]
        pre_series=return_series.iloc[:day0]
        vol= return_series.iloc[day0-min(len(pre_series),vol_lookback+1):day0].std()
        ret=(return_series.iloc[day0:day0+EAR_period]+1).prod()-1
        nmove=ret/vol
        return nmove
    else:
        return None

In [ ]:
def signal_vol(signal_column,return_df,vol_lookback):
    '''
    Calculate simple vol from signal tuple
    '''
    signal_series=return_df.loc[signal_column.name[0]]
    location=signal_series.index.tolist().index(signal_column.name[1])
    vol_range=min(vol_lookback,len(signal_series[:location]))
    signal_vol=signal_series[location-vol_range-1:location].std()
    return signal_vol

In [ ]:
def slice_universe(signal_df,start_datetime,end_datetime,old_position):
    '''
    Slice the signal_df, both the index and entry date have to be 
    '''
    

    signal_df=signal_df.loc[start_datetime:end_datetime]
    
    if old_position is True:  
        adj_signal_df=signal_df
    else:
        entry=signal_df.apply(lambda x:datetime.strptime(x.name[1],"%d/%b/%Y"),axis=0)
        period_evaluate=(entry>=start_datetime)&(entry<=end_datetime)
        adj_signal_df=signal_df.loc[:,period_evaluate]
    
    
    zero_index=pd.Series(1,index=pd.date_range(start_datetime,end_datetime,freq='B')).to_frame()
    adj_signal_df=pd.concat([adj_signal_df.drop_duplicates(),zero_index],axis=1).iloc[:,:-1]
    return adj_signal_df
    

In [ ]:
def signal_filter_stop(signal_df,stop_level,return_df,vol_lookback,stop_type,index_df):
    '''
    Input - signal_df
    Get the updated signal df after the stop loss
    stop_type:abs,rel
    
    '''
    if stop_type=='abs':
        vol_row=signal_df.apply(lambda column:signal_vol(column,return_df,vol_lookback),axis=0)
        signal_cum_nmove=((1+signal_df).cumprod()-1).ffill()/vol_row
        signal_df_stop=signal_df[-(signal_cum_nmove.expanding().min().shift(1,axis=0)<-stop_level)]
    elif stop_type=='rel':
        if index_df.shape[1]==1:
            signal_count=signal_df.copy()
            signal_count[((signal_count)>0) | ((signal_count)<0)]=1.0
            signal_hedge=signal_count.apply(lambda x:x.multiply(index_df.iloc[:,0],axis=0))
            
            vol_row=signal_df.apply(lambda x:signal_vol(x,return_df,vol_lookback),axis=0)
            rel_signal_cum_nmove=((1+signal_df).cumprod()-(1+signal_hedge).cumprod()).ffill()/vol_row
            signal_df_stop=signal_df[-(rel_signal_cum_nmove.expanding().min().shift(1,axis=0)<-stop_level)]
        else:
            signal_count=signal_df.copy()
            signal_count[((signal_count)>0) | ((signal_count)<0)]=1.0
            signal_hedge=signal_count.apply(lambda x:x.multiply(index_df[Asia_mapping.loc[x.name[0][-2:]].iloc[0]],axis=0))
            
            vol_row=signal_df.apply(lambda x:signal_vol(x,return_df,vol_lookback),axis=0)
            rel_signal_cum_nmove=((1+signal_df).cumprod()-(1+signal_hedge).cumprod()).ffill()/vol_row
            signal_df_stop=signal_df[-(rel_signal_cum_nmove.expanding().min().shift(1,axis=0)<-stop_level)]            
            
    else:
        pass
        
    return signal_df_stop

In [ ]:
def revision_adjusted_size(reference_signal_df,lower_revision,higher_revision,size_multiple,revision_row,revision_row_reference,gross,long):
    
    ''' 
    Use positive size
    '''
    
    lower_size=0.01
    higher_size=lower_size*size_multiple
    
    if long is True:
        
        size_row_reference=revision_row_reference.to_frame().copy().apply(lambda x: lower_size+(higher_size-lower_size)\
                                                      *(np.abs(x.iloc[0]-lower_revision))/np.abs(higher_revision-lower_revision) \
                                                      if np.abs(x.iloc[0])<=np.abs(higher_revision) else higher_size,axis=1)

        size_df_reference=(1+reference_signal_df).cumprod()*size_row_reference

        trial_gross=np.abs(size_df_reference.sum(axis=1).mean())
        new_lower_size=lower_size/(trial_gross*100/gross)
        new_higher_size=higher_size/(trial_gross*100/gross)

        size_row=revision_row.to_frame().copy().apply(lambda x: new_lower_size+(new_higher_size-new_lower_size)\
                                                      *(np.abs(x.iloc[0]-lower_revision))/np.abs(higher_revision-lower_revision) \
                                                      if np.abs(x.iloc[0])<=np.abs(higher_revision) else new_higher_size,axis=1)
    else:
        size_row_reference=revision_row_reference.to_frame().copy().apply(lambda x: lower_size+(higher_size-lower_size)\
                                                      *(np.abs(x.iloc[0]-lower_revision))/np.abs(higher_revision-lower_revision) \
                                                      if np.abs(x.iloc[0])<=np.abs(lower_revision) else higher_size,axis=1)

        size_df_reference=(1-reference_signal_df).cumprod()*size_row_reference

        trial_gross=np.abs(size_df_reference.sum(axis=1).mean())
        new_lower_size=lower_size/(trial_gross*100/gross)
        new_higher_size=higher_size/(trial_gross*100/gross)

        size_row=revision_row.to_frame().copy().apply(lambda x: new_lower_size+(new_higher_size-new_lower_size)\
                                                      *(np.abs(x.iloc[0]-lower_revision))/np.abs(higher_revision-lower_revision) \
                                                      if np.abs(x.iloc[0])<=np.abs(lower_revision) else new_higher_size,axis=1)

    return size_row, new_lower_size

In [ ]:
def sizing(signal_df,reference_signal_df,gross,fundamental_df,new_signal,return_df,risk_parity,liquidity,capital,\
           revision_adjust,long):
    '''
    Use historical signal_df range to calculate the size row for the current signal_df range
    Idea is to use historical as a benchmark for future sizing
    '''
    
    fundamental_df=fundamental_df.copy().sort_index()
    vol_reference=reference_signal_df.apply(lambda x:signal_vol(x,return_df,30),axis=0).mean()
    vol_row=signal_df.apply(lambda x:signal_vol(x,return_df,30),axis=0)
    
    '''
    Revision row needs to be updated using reference_signal
    new sizing scheme is a linear function
    revision_adjust=(True/False,lower_revision,higher_revision,size_multiple)
    '''
    
    if revision_adjust[0] is True:
        
        if new_signal is True:     
            revision_row_reference=reference_signal_df.apply(lambda x:fundamental_df.loc[x.name,"Revision_real"],axis=0)
            revision_row=signal_df.apply(lambda x:fundamental_df.loc[x.name,"Revision_real"],axis=0)
        else:
            revision_row_reference=reference_signal_df.apply(lambda x:fundamental_df.loc[x.name,"Revision_20"],axis=0)
            revision_row=signal_df.apply(lambda x:fundamental_df.loc[x.name,"Revision_20"],axis=0)            
        
        if long is True and revision_adjust[1] is not None:
            lower_revision=revision_adjust[1][0]
            higher_revision=revision_adjust[1][1]
            size_multiple=revision_adjust[3]
        
            base_size,low_size=revision_adjusted_size(reference_signal_df,lower_revision,higher_revision,size_multiple,\
                                                      revision_row,revision_row_reference,gross,True)
            if risk_parity is True:
                size_row=signal_df.apply(lambda x: min(base_size[x.name]/(vol_row[x.name]/vol_reference),\
                                                       fundamental_df.loc[x.name[0],x.name[1]]["Volume"].iloc[0]*\
                                                       liquidity/capital),axis=0)
            else:
                size_row=signal_df.apply(lambda x: min(base_size[x.name], fundamental_df.loc[x.name[0],x.name[1]]["Volume"].iloc[0]\
                                                       *liquidity/capital),axis=0)            

        elif long is False and revision_adjust[2] is not None:
            lower_revision=revision_adjust[2][0]
            higher_revision=revision_adjust[2][1]
            size_multiple=revision_adjust[3]
        
            base_size,low_size=revision_adjusted_size(reference_signal_df,lower_revision,higher_revision,size_multiple,revision_row,\
                                             revision_row_reference,gross,False) 

            if risk_parity is True:
                size_row=signal_df.apply(lambda x: min(base_size[x.name]/(vol_row[x.name]/vol_reference),\
                                                       fundamental_df.loc[x.name[0],x.name[1]]["Volume"].iloc[0]*\
                                                       liquidity/capital),axis=0)
            else:
                size_row=signal_df.apply(lambda x: min(base_size[x.name], fundamental_df.loc[x.name[0],x.name[1]]["Volume"].iloc[0]\
                                                       *liquidity/capital),axis=0)

        else:
            size_row=None
            low_size=None
        

    else:
        number=reference_signal_df.count(axis=1).mean()
        avg_size=gross/100/number

        if risk_parity is True:
            size_row=signal_df.apply(lambda x: min(avg_size/(vol_row[x.name]/vol_reference),\
                                                   fundamental_df.loc[x.name[0],x.name[1]]["Volume"].iloc[0]*\
                                                   liquidity/capital),axis=0)
        else:
            size_row=signal_df.apply(lambda x: min(avg_size, fundamental_df.loc[x.name[0],x.name[1]]["Volume"].iloc[0]\
                                                   *liquidity/capital),axis=0)
        low_size=None
    return size_row,low_size

In [ ]:
def trading_analytics_date(portfolio_cache):
    '''
    Key portfolio metrics from portfolio cache
    Feed into plot function
    '''
    
    ind_return=portfolio_cache[3]
    signal_count=len(ind_return)
    account_curve=portfolio_cache[1]
    
    if signal_count==0:
        return None,None,None,None,None,None,None
    else:
        mean_return=ind_return.mean()
        hit_rate=len(ind_return[ind_return>0])/len(ind_return)*1.0
        payoff_ratio=ind_return[ind_return>0].mean()/ind_return[ind_return<0].mean()*-1.0
        
        account_price=account_curve+1
        ann_vol=np.std(account_price.diff()/account_price.shift(1))*(260**0.5)
        ann_ret=(account_price.iloc[-1]**(1/len(account_price)))**260-1
        ann_sharpe=ann_ret/ann_vol
        
        max_dd=-((1+account_curve)-(1+account_curve).cummax(axis=0)).expanding().min().min()
        
        #low_date=(np.maximum.accumulate(account_curve)-account_curve).idxmax()
        #high_date=account_curve[:low_date].idxmax()
        #max_dd=1-(1+account_curve[low_date])/(1+account_curve[high_date])
        
        return signal_count,hit_rate,payoff_ratio,ann_ret,ann_vol,ann_sharpe,max_dd

In [ ]:
def trading_analytics_simp(account_curve):
    '''
    Key portfolio metrics from portfolio account curve
    Only sharpe and drawdown
    '''


    account_price=account_curve+1
    ann_vol=np.std(account_price.diff()/account_price.shift(1))*(260**0.5)
    ann_ret=(account_price.iloc[-1]**(1/len(account_price)))**260-1
    ann_sharpe=ann_ret/ann_vol

    max_dd=-((1+account_curve)-(1+account_curve).cummax(axis=0)).expanding().min().min()

    #low_date=(np.maximum.accumulate(account_curve)-account_curve).idxmax()
    #high_date=account_curve[:low_date].idxmax()
    #max_dd=1-(1+account_curve[low_date])/(1+account_curve[high_date])

    return ann_sharpe,max_dd

In [ ]:
def plot_signal(title,figsize,portfolio_cache):

    account_curve=portfolio_cache[1]
    avg_size=np.abs(portfolio_cache[2]).mean(axis=0).mean()
    ind_return=portfolio_cache[3]
    gross=portfolio_cache[4]
    turnover=portfolio_cache[5]

    fig=plt.figure(figsize=figsize)
    ax1=fig.add_subplot(1,1,1)
    ln1=ax1.plot(account_curve,label='signal',color='b')

    val1=ax1.get_yticks()
    start=val1[0]
    end=val1[-1]
    ax1.set_yticks(np.arange(start,end,0.01))  
    adj_val1=ax1.get_yticks()
    ax1.set_yticklabels(["{:.0%}".format(x) for x in adj_val1])

    ax2=ax1.twinx()
    ln2=ax2.plot(gross,label='gross',color='silver')

    val2=ax2.get_yticks()
    start=val2[0]
    end=val2[-1]
    ax2.set_yticks(np.arange(start,end,0.1))  
    adj_val2=ax2.get_yticks()
    ax2.set_yticklabels(["{:.0%}".format(x) for x in adj_val2])

    count,hit,payoff,ret,vol,sharpe,max_dd=trading_analytics_date(portfolio_cache)

    plt.title("\n".join(wrap('count='+str(count)+
                             ',avg_size='+str("{:.1%}".format(avg_size))+
                             ',hit_rate='+str("{:.0%}".format(hit))+
                             ',payoff='+str(round(payoff,1))+
                             ',return='+str("{:.1%}".format(ret))+
                             ',vol='+str("{:.1%}".format(vol))+
                             ',sharpe='+str(round(sharpe,1))+
                             ',turnover='+str(round(turnover,1))+'x'+                             
                             ',max_drawdown='+str("{:.1%}".format(max_dd)))),fontsize=10)

    ax1.set_xlabel('Year')
    ax1.set_ylabel('Return')
    ax2.set_ylabel('Exposure')
    plt.suptitle(title,y=1.05,fontsize=16)
    plt.grid(linestyle='dashed')
    plt.legend(ln1+ln2,[l.get_label() for l in ln1+ln2],loc=2)
    ax1.axhline(y=0,color='k')

    plt.show()        

# 4. Signal class 

In [18]:
class signal(object):
    '''
    Signal class is built to initialize the signal_df and account curve from base parameters 
    '''
    def __init__(self,fundamental_df,price_df,EAR_period,entry,long_criteria,short_criteria,holding,start,end,old_position\
                 ,new_signal,revision_adjust,early_exit):
        '''
        Define the key free parameters of the signal
        Criteria:(EAR,revision,revision_norm,size)
        revision_adjust:(True/False,(long lower revision,long higher revision),(short low abs revision,short high abs_revision),size_multiple)
        '''
        self.fundamental_df=fundamental_df
        self.price_df=price_df
        self.abs_return=price_df.diff(1,axis=1)/price_df.shift(1,axis=1)
        self.EAR_period=EAR_period
        self.entry=entry
        self.long_criteria=long_criteria
        self.short_criteria=short_criteria
        self.holding=holding
        self.start=start
        self.end=end
        self.old_position=old_position
        self.new_signal=new_signal
        self.revision_adjust=revision_adjust #(True/False,lower_revision,higher_revision in absolute terms,size_multiple)
        self.early_exit=early_exit
        
    def signal_base(self):
        '''
        Filter the signal criteria like EAR and revision, get the target signal list
        From the fundamental information
        For both long and short side
        '''
        
        long_base=self.fundamental_df.copy()
        short_base=self.fundamental_df.copy()
        

        long_base["EAR"]=long_base.apply(lambda x:EAR_calc(x.name[0],x.name[1],self.abs_return,self.EAR_period,30)\
                                     if x.name[0] in self.abs_return.index else None,axis=1)

        short_base["EAR"]=short_base.apply(lambda x:EAR_calc(x.name[0],x.name[1],self.abs_return,self.EAR_period,30)\
                                     if x.name[0] in self.abs_return.index else None,axis=1)


        if self.long_criteria is None:
            long_base=None
        else:
            
            if self.long_criteria[1] is None:
                pass
            else:
                if self.new_signal is True:
                    long_base=long_base[(long_base["Revision_real"]>self.long_criteria[1][0])\
                                                    &(long_base["Revision_real"]<self.long_criteria[1][1])]
                else:
                    long_base=long_base[(long_base["Revision_20"]>self.long_criteria[1][0])\
                                                    &(long_base["Revision_20"]<self.long_criteria[1][1])]                    

            if self.long_criteria[2] is None:
                pass
            else:
                long_base=long_base[(long_base["Revision_norm"]>self.long_criteria[2][0])\
                                                    &(long_base["Revision_norm"]<self.long_criteria[2][1])]
                
            if self.long_criteria[3] is None:
                pass
            else:
                long_base=long_base[(long_base["Market cap"]>self.long_criteria[3][0])&\
                                        (long_base["Market cap"]<self.long_criteria[3][1])]
            
            if self.long_criteria[0] is None:
                pass
            else:
                long_base=long_base[(long_base["EAR"]>self.long_criteria[0][0])&(long_base["EAR"]<self.long_criteria[0][1])]
                
                
        if self.short_criteria is None:
            short_base=None
        else:
            
            if self.short_criteria[1] is None:
                pass
            else:
                if self.new_signal is True:
                    short_base=short_base[(short_base["Revision_real"]>self.short_criteria[1][0])\
                                                    &(short_base["Revision_real"]<self.short_criteria[1][1])]
                else:
                    short_base=short_base[(short_base["Revision_20"]>self.short_criteria[1][0])\
                                                    &(short_base["Revision_20"]<self.short_criteria[1][1])]                    

            if self.short_criteria[2] is None:
                pass
            else:
                short_base=short_base[(short_base["Revision_norm"]>self.short_criteria[2][0])\
                                                    &(short_base["Revision_norm"]<self.short_criteria[2][1])]
                
            if self.short_criteria[3] is None:
                pass
            else:
                short_base=short_base[(short_base["Market cap"]>self.short_criteria[3][0])&\
                                        (short_base["Market cap"]<self.short_criteria[3][1])]
            
            if self.short_criteria[0] is None:
                pass
            else:
                short_base=short_base[(short_base["EAR"]>self.short_criteria[0][0])&\
                                      (short_base["EAR"]<self.short_criteria[0][1])]
        
        self.long_base=long_base
        self.short_base=short_base
        
        return long_base,short_base
    
    def signal_df_date(self):#if we hold them through next earning
        '''
        Obtain the signal_df function over the whole time period from the target signal list
        '''
        try:
            long_base=self.long_base.copy()
            short_base=self.short_base.copy()
        except:
            long_base,short_base=signal.signal_base(self)
        
        if long_base is None:
            long_df=None
        
        else:
            long_df=pd.DataFrame(index=self.price_df.columns)

            for s in long_base.index:
                return_series=self.abs_return.loc[s[0]]
                if s[1] in return_series.index:
                    if not np.isnan(return_series.loc[s[1]]): 
                        day0=return_series.index.tolist().index(s[1])
                        
                        if self.early_exit is True and datetime.strptime(s[1],"%d/%b/%Y").date()>=\
                        datetime.strptime(s[3],"%d/%b/%Y").date():
                            period=None
                        
                        elif self.early_exit is True and type(s[2])!=float and datetime.strptime(s[1],"%d/%b/%Y").date()<\
                        datetime.strptime(s[3],"%d/%b/%Y").date():
                            period=min(self.holding,np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[2],"%d/%b/%Y").date())-self.entry,
                                      np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[3],"%d/%b/%Y").date())-self.entry+1)                            
                        
                        elif self.early_exit is True and type(s[2])==float and datetime.strptime(s[1],"%d/%b/%Y").date()<\
                        datetime.strptime(s[3],"%d/%b/%Y").date():
                            period=min(self.holding,np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[3],"%d/%b/%Y").date())-self.entry+1)
                            
                        elif type(s[2])==float:##basically np.nan has type float
                            period=self.holding+1
                        
                        else: ##assume that we are not holding through numbers
                            period=min(self.holding,np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[2],"%d/%b/%Y").date())-self.entry)
                            
                        if period is None:
                            pass
                        else:
                            target_series=return_series.iloc[day0+self.entry-1:day0+min(period+self.entry, \
                                                                                           len(return_series[day0:]))].dropna()
                            
                            if len(target_series)==0:
                                pass
                            else:
                                target_series.iloc[0]=0.0
                                long_df[s]=target_series                        

                    else:
                        pass
                else:
                    pass
                
            long_df=long_df.reindex(datetime.strptime(i,"%d/%b/%Y") for i in long_df.index)
            long_df=long_df.sort_index()

            if self.start is not None:
                long_df=slice_universe(long_df,self.start,self.end,self.old_position)
            else:
                pass
            
            long_df=long_df.dropna(how="all",axis=1)
            #long_df.columns=pd.MultiIndex.from_tuples(pd.Series(list(long_df.columns)))
            self.long_df=long_df
            
            
        
        if short_base is None:
            short_df=None
        
        else:
            short_df=pd.DataFrame(index=self.price_df.columns)

            for s in short_base.index:
                return_series=self.abs_return.loc[s[0]]
                if s[1] in return_series.index:
                    if not np.isnan(return_series.loc[s[1]]): 
                        day0=return_series.index.tolist().index(s[1])
                        
                        if self.early_exit is True and datetime.strptime(s[1],"%d/%b/%Y").date()>=\
                        datetime.strptime(s[3],"%d/%b/%Y").date():
                            period=None
                            
                        elif self.early_exit is True and type(s[2])!=float and datetime.strptime(s[1],"%d/%b/%Y").date()<\
                        datetime.strptime(s[3],"%d/%b/%Y").date():
                            period=min(self.holding,np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[2],"%d/%b/%Y").date())-self.entry,
                                      np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[3],"%d/%b/%Y").date())-self.entry+1)  
                            
                        elif self.early_exit is True and type(s[2])==float and datetime.strptime(s[1],"%d/%b/%Y").date()<\
                        datetime.strptime(s[3],"%d/%b/%Y").date():
                            period=min(self.holding,np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[3],"%d/%b/%Y").date())-self.entry+1)
                        elif type(s[2])==float:##basically np.nan has type float
                            period=self.holding
                    
                        else: ##assume that we are not holding through numbers
                            period=min(self.holding,np.busday_count(datetime.strptime(s[1],"%d/%b/%Y").date(),\
                                                                    datetime.strptime(s[2],"%d/%b/%Y").date())-self.entry)\


                        if period is None:
                            pass
                        else:
                            target_series=return_series.iloc[day0+self.entry-1:day0+min(period+self.entry, \
                                                                                           len(return_series[day0:]))].dropna()
                            
                            if len(target_series)==0:
                                pass
                            else:
                                target_series.iloc[0]=0.0
                                short_df[s]=target_series         
                            
                    else:
                        pass
                else:
                    pass

            short_df=short_df.reindex(datetime.strptime(i,"%d/%b/%Y") for i in short_df.index)
            short_df=short_df.sort_index()

            if self.start is not None:
                short_df=slice_universe(short_df,self.start,self.end,self.old_position)
            else:
                pass  
            
            short_df=short_df.dropna(how="all",axis=1)
            #short_df.columns=pd.MultiIndex.from_tuples(pd.Series(list(short_df.columns)))
            self.short_df=short_df
        
        return long_df,short_df

    def signal_account(self,stop,gross,index_df,net_level,risk_parity,liquidity,capital):
        '''
        Build the account curve with signal_df
        Assume quarterly rebalancing that's why the period list has quarter as the key
        Take extra care when building the account curve, the logic is: work out the size_df, then shift by 1 and * signal_df
        Stop=(long_stop,short_stop,type)
        index_df has to be a dataframe with a name
        '''
        
        try:
            long_df=self.long_df.copy()
            short_df=self.short_df.copy()
            
        except:
            long_df,short_df=self.signal_df_date()
                   
        '''
        Assign values for later use
        '''
        
        self.capital=capital
        
        self.index_df=index_df
        self.index_df.index=[datetime.strptime(i,"%d/%b/%Y") for i in self.index_df.index]
       
        
        '''
        Define rebalance period first
        '''
        if long_df is None:
            period_quarter=short_df.apply(lambda x:str(x.name.year)+" "+str(x.name.quarter),axis=1)
            period_list=list(set(period_quarter))
            period_list.sort()    
            
        else:
            period_quarter=long_df.apply(lambda x:str(x.name.year)+" "+str(x.name.quarter),axis=1)
            period_list=list(set(period_quarter))
            period_list.sort()    
                
        '''
        Separate out long and short
        '''
        if long_df is None:
            long_cache=(None,None,None,None)
        else:
            if stop is None:
                pass
            else:
                long_df=signal_filter_stop(long_df,stop[0],self.abs_return,30,stop[2],self.index_df)   
                self.long_df=long_df

            long_sub_signal={}
            long_sub_size_row={}
            long_sub_size_df={}
            long_sub_pnl={}
        
            for s in period_list:
            
                long_sub_signal[s]=long_df[period==s].dropna(how='all',axis=1)
                
                if long_sub_signal[s].shape[1]==0:
                    long_sub_size_df[s]=long_sub_signal[s]
                    long_sub_pnl[s]=long_sub_signal[s]
                    
                else:
                    if period_list.index(s)<4:##use last quarter's sizing as reference
                        long_sub_size_row[s]=sizing(long_sub_signal[s],long_sub_signal[s],gross[0],self.fundamental_df,\
                                                       self.new_signal,self.abs_return,risk_parity,liquidity,capital,\
                                                    self.revision_adjust,True)[0]
                    else:
                        try:
                            long_sub_size_row[s]=sizing(long_sub_signal[s],long_sub_signal[period_list[period_list.index(s)-1]],\
                                                       gross[0],self.fundamental_df,self.new_signal,self.abs_return,\
                                                        risk_parity,liquidity,capital,self.revision_adjust,True)[0]
                        except:
                            long_sub_size_row[s]=sizing(long_sub_signal[s],long_sub_signal[s],\
                                                       gross[0],self.fundamental_df,self.new_signal,self.abs_return,\
                                                        risk_parity,liquidity,capital,self.revision_adjust,True)[0] 

                    long_sub_size_df[s]=(1+long_sub_signal[s]).cumprod()*long_sub_size_row[s]
                    long_sub_pnl[s]=(long_sub_size_df[s].shift(1))*long_sub_signal[s] 
                    # need to shift by 1 as the size is end of the day
        
            long_daily_pnl=pd.concat(list(long_sub_pnl.values()),axis=0)
            long_acct_curve=long_daily_pnl.cumsum().ffill().sum(axis=1)
            long_size_df=pd.concat(list(long_sub_size_df.values()),axis=0)
            long_ind_return=long_daily_pnl.cumsum().ffill().iloc[-1].dropna()
            long_cache=(long_daily_pnl,long_acct_curve,long_size_df,long_ind_return)
            
            self.long_cache=long_cache

            
        if short_df is None:
            short_cache=(None,None,None,None)
        else:
            if stop is None:
                pass
            else:
                short_df=-signal_filter_stop(-short_df,stop[1],self.abs_return,30,stop[2],self.index_df)   
                self.short_df=short_df

            short_sub_signal={}
            short_sub_size_row={}
            short_sub_size_df={}
            short_sub_pnl={}
        
            for s in period_list:
                short_sub_signal[s]=short_df[period==s].dropna(how='all',axis=1)

                if short_sub_signal[s].shape[1]==0:
                    short_sub_size_df[s]=short_sub_signal[s]
                    short_sub_pnl[s]=short_sub_signal[s]
                    
                else:
                    
                    if period_list.index(s)<4:##use last quarter's sizing as reference
                        short_sub_size_row[s]=-sizing(short_sub_signal[s],short_sub_signal[s],gross[1],self.fundamental_df,\
                                                       self.new_signal,self.abs_return,risk_parity,liquidity,capital,\
                                                      self.revision_adjust,False)[0]
                    else:
                        try:
                            short_sub_size_row[s]=-sizing(short_sub_signal[s],short_sub_signal\
                                                          [period_list[period_list.index(s)-1]],\
                                                           gross[1],self.fundamental_df,self.new_signal,self.abs_return,\
                                                          risk_parity,liquidity,\
                                                          capital,self.revision_adjust,False)[0]
                        except:
                            short_sub_size_row[s]=-sizing(short_sub_signal[s],short_sub_signal[s],\
                                                           gross[1],self.fundamental_df,self.new_signal,self.abs_return,\
                                                          risk_parity,liquidity,\
                                                          capital,self.revision_adjust,False)[0]

                    short_sub_size_df[s]=(1+short_sub_signal[s]).cumprod()*short_sub_size_row[s]
                    short_sub_pnl[s]=(short_sub_size_df[s].shift(1))*short_sub_signal[s] 
                # need to shift by 1 as the size is end of the day
        
            short_daily_pnl=pd.concat(list(short_sub_pnl.values()),axis=0)
            short_acct_curve=short_daily_pnl.cumsum().ffill().sum(axis=1)
            short_size_df=pd.concat(list(short_sub_size_df.values()),axis=0)
            short_ind_return=short_daily_pnl.cumsum().ffill().iloc[-1].dropna()
            
            short_cache=(short_daily_pnl,short_acct_curve,short_size_df,short_ind_return)
            self.short_cache=short_cache
    
        '''Put alpha positions together to form the alpha part'''
        alpha_df=pd.concat([long_df,short_df],axis=1)
        self.alpha_df=alpha_df
        
        alpha_daily_pnl=pd.concat([long_cache[0],short_cache[0]],axis=1)
        alpha_acct_curve=alpha_daily_pnl.cumsum().ffill().sum(axis=1)
        alpha_size_df=pd.concat([long_cache[2],short_cache[2]],axis=1)
        alpha_ind_return=pd.concat([long_cache[3],short_cache[3]],axis=0)
        
        alpha_cache=(alpha_daily_pnl,alpha_acct_curve,alpha_size_df,alpha_ind_return)
        
        
        if self.index_df is not None:
            if self.index_df.shape[1]==1:
                index_df=self.index_df.copy().loc[alpha_df.index] 
                index_size_df=(net_level-alpha_size_df.sum(axis=1)).to_frame(index_df.columns[0])
                index_daily_pnl=index_size_df.shift(1)*index_df
                index_acct_curve=index_daily_pnl.cumsum()
                index_ind_return=index_acct_curve.iloc[-1]
                index_cache=(index_daily_pnl,index_acct_curve,index_size_df,index_ind_return)
            else:
                index_df=self.index_df.copy().loc[alpha_df.index] 
                alpha_temp=alpha_cache[2].copy().T
                alpha_temp["index"]=alpha_temp.apply(lambda x:Asia_mapping.loc[x.name[0][-2:]].iloc[0],axis=1)
                index_size_df=net_level-alpha_temp.groupby("index").apply(sum).T.iloc[:-1]
                index_daily_pnl=index_size_df.shift(1)*index_df
                index_acct_curve=index_daily_pnl.cumsum()
                index_ind_return=index_acct_curve.iloc[-1]
                index_cache=(index_daily_pnl,index_acct_curve,index_size_df,index_ind_return)
        else:
            index_cache=(None,None,None,None)
            
        '''Finally put everything together'''    
        portfolio_df=pd.concat([alpha_df,index_df],axis=1)
            
        portfolio_size_df=pd.concat([alpha_cache[2],index_cache[2]],axis=1).sort_index()
  
        portfolio_daily_pnl=pd.concat([alpha_cache[0],index_cache[0]],axis=1).sort_index()

        portfolio_acct_curve=portfolio_daily_pnl.cumsum().ffill().sum(axis=1)
        portfolio_ind_return=alpha_cache[3].copy()
        
        portfolio_gross=np.abs(portfolio_size_df).sum(axis=1).sort_index()
        portfolio_turnover=(np.abs(alpha_size_df.fillna(0.0).diff(1)).sum().sum())/(portfolio_size_df.shape[0]/260)
        
        portfolio_cache=(portfolio_daily_pnl,portfolio_acct_curve,portfolio_size_df,portfolio_ind_return,portfolio_gross,\
                         portfolio_turnover,portfolio_df)
        
        self.portfolio_account=portfolio_cache #save for later use
        
        
        return long_cache,short_cache,alpha_cache,portfolio_cache
    
    def plot_account(self,title,figsize=[10,4],portfolio=None):
        '''
        Plot the account curve
        '''
        if portfolio is None:
            try:
                portfolio_cache=self.portfolio_account

            except AttributeError:
                print("Execute the signal_account first!")  
                return None
        else:
            portfolio_cache=portfolio
        
        plot_signal(title,figsize,portfolio_cache)




In [ ]:
def combine_signal(portfolio_list,capital_list):
    
    
    daily_pnl_list=[]
    signal_df_list=[]
    portfolio_size_list=[]
    for i in range(len(portfolio_list)):

        daily_pnl_list.append(portfolio_list[i][0]*capital_list[i]/np.sum(capital_list))
        signal_df_list.append(portfolio_list[i][-1]*capital_list[i]/np.sum(capital_list))
        portfolio_size_list.append(portfolio_list[i][2]*capital_list[i]/np.sum(capital_list))
    
    daily_pnl=pd.concat(daily_pnl_list,axis=1)
    signal_df=pd.concat(signal_df_list,axis=1)
    size_df=pd.concat(portfolio_size_list,axis=1)
    
    account_curve=daily_pnl.cumsum().ffill().sum(axis=1)
    ind_return=daily_pnl.cumsum().ffill().iloc[-1]

    gross=np.abs(size_df).sum(axis=1)
    turnover=(np.abs(size_df.fillna(0.0).diff(1)).sum().sum())/(size_df.shape[0]/260)
    
    portfolio_cache=(daily_pnl,account_curve,size_df,ind_return,gross,turnover,signal_df)
    return portfolio_cache

In [ ]:
def sharpe_by_year(account_curve,year_list):
    result_dict={}
    for i in year_list:
        sub_curve=account_curve.iloc[(account_curve.index>=pd.Timestamp(i,1,1))&(account_curve.index<=pd.Timestamp(i,12,31))]
        result_dict[i]=trading_analytics_simp(sub_curve)[0]
    return result_dict

In [ ]:
def dd_by_year(account_curve,year_list):
    result_dict={}
    for i in year_list:
        sub_curve=account_curve.iloc[(account_curve.index>=pd.Timestamp(i,1,1))&(account_curve.index<=pd.Timestamp(i,12,31))]
        result_dict[i]=trading_analytics_simp(sub_curve)[1]
    return result_dict

# 5 - Result

## 5.1-Europe

In [ ]:
Europe_new=signal(fundamental_df=Europe2_old,
                   price_df=Europe_VWAP,
                   EAR_period=1,
                   entry=3,
                   long_criteria=((2,1000),(0.02,1000),None,None),
                   short_criteria=((-1000,-2),(-1000,-0.02),None,(5000,10000000)),
                   holding=30,
                   start=pd.Timestamp(2010,1,1),
                   end=pd.Timestamp(2019,3,29),
                   old_position=False,
                   new_signal=True,
                   revision_adjust=(True,(0.02,0.05),(-0.05,-0.02),2),
                  early_exit=False)

In [ ]:
_long_Europe,_short_Europe,_alpha_Europe,_portfolio_Europe=Europe_new.signal_account(stop=[9,12,"rel"],
                                                           gross=(26,13),
                                                           index_df=abs_return_index_Europe.loc["SX5E Index"].to_frame('Europe'),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=60)

In [ ]:
plot_signal("Europe new signal - no stop",[10,4],Europe_new.portfolio_account)

In [ ]:
plot_signal("Europe new signal - 100k abs stop",[10,4],Europe_new.portfolio_account)

In [ ]:
plot_signal("Europe new signal - 200k abs stop",[10,4],Europe_new.portfolio_account)

In [ ]:
plot_signal("Europe new signal - 100k rel stop",[10,4],Europe_new.portfolio_account)

In [ ]:
plot_signal("Europe new signal - 200k rel stop",[10,4],Europe_new.portfolio_account)

In [ ]:
Europe_orig=signal(fundamental_df=Europe2_old,
                    price_df=Europe_price,
                    EAR_period=1,
                    entry=2,
                    long_criteria=((1,1000),(0.03,1000),None,None),
                    short_criteria=((-1000,-2),(-1000,-0.03),None,(5000,10000000)),
                    holding=30,
                    start=pd.Timestamp(2010,1,1),
                    end=pd.Timestamp(2019,3,29),
                    old_position=False,
                    new_signal=False,
                    revision_adjust=(False,0.02,0.1,2),
                    early_exit=False)

In [ ]:
_long_Europe,_short_Europe,_alpha_Europe,_portfolio_Europe=Europe_orig.signal_account(stop=[4.5,6,"rel"],
                                                           gross=(25,13),
                                                           index_df=abs_return_index_Europe.loc["SX5E Index"].to_frame('Europe'),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=60)

In [ ]:
plot_signal("Europe orig signal - no stop",[10,4],Europe_orig.portfolio_account)

In [ ]:
plot_signal("Europe orig signal - abs stop 100k",[10,4],Europe_orig.portfolio_account)

In [ ]:
plot_signal("Europe orig signal - abs stop 200k",[10,4],Europe_orig.portfolio_account)

In [ ]:
plot_signal("Europe orig signal - rel stop 100k",[10,4],Europe_orig.portfolio_account)

In [ ]:
plot_signal("Europe orig signal - rel stop 200k",[10,4],Europe_orig.portfolio_account)

## 5.2 - Asia

In [ ]:
Asia_new=signal(fundamental_df=Asia2_old,
                price_df=Asia_VWAP,
                EAR_period=2,
                entry=3,
                long_criteria=((1,1000),(0.02,1000),None,None),
                short_criteria=None,
                holding=20,
                start=pd.Timestamp(2010,1,1),
                end=pd.Timestamp(2019,3,29),
                old_position=False,
                new_signal=True,
                revision_adjust=(True,(0.02,0.05),(-0.1,-0.02),2),
                early_exit=False)

In [ ]:
_long_Asia,_short_Asia,_alpha_Asia,_portfolio_Asia=Asia_new.signal_account(stop=[5.5,None,'rel'],
                                                           gross=(28,None),
                                                           index_df=abs_return_index_Asia.loc[["AS51 Index","HSI Index",
                                                                                                "TPX Index","KOSPI Index",
                                                                                                "MXSG Index","TAMSCI Index"]].T,
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=40)

In [ ]:
Asia_new.plot_account("Asia new signal - no stop")

In [ ]:
Asia_new.plot_account("Asia new signal - 100k abs stop")

In [ ]:
Asia_new.plot_account("Asia new signal - 200k abs stop")

In [ ]:
Asia_new.plot_account("Asia new signal - 100k rel stop")

In [ ]:
Asia_new.plot_account("Asia new signal - 200k rel stop")

In [ ]:
Asia_orig=signal(fundamental_df=Asia2_old,
                 price_df=Asia_price,
                 EAR_period=2,
                 entry=2,
                 long_criteria=((2,1000),(0.02,1000),None,None),
                 short_criteria=None,
                 holding=20,
                 start=pd.Timestamp(2010,1,1),
                 end=pd.Timestamp(2019,3,29),
                 old_position=False,
                 new_signal=False,
                 revision_adjust=(False,(0.02,0.1),(-0.1,-0.02),3),
                 early_exit=False)

In [ ]:
_long_Asia,_short_Asia,_alpha_Asia,_portfolio_Asia=Asia_orig.signal_account(stop=[16,None,'rel'],
                                                           gross=(29,None),
                                                           index_df=abs_return_index_Asia.loc[["AS51 Index","HSI Index",
                                                                                                "TPX Index","KOSPI Index",
                                                                                                "MXSG Index","TAMSCI Index"]].T,
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=40)

In [ ]:
Asia_orig.plot_account("Asia orig signal - no stop")

In [ ]:
Asia_orig.plot_account("Asia orig signal - 100k abs stop")

In [ ]:
Asia_orig.plot_account("Asia orig signal - 200k abs stop")

In [ ]:
Asia_orig.plot_account("Asia orig signal - 100k rel stop")

In [ ]:
Asia_orig.plot_account("Asia orig signal - 200k rel stop")

## 5.3 - US

In [ ]:
US_new=signal(fundamental_df=US1_old,
               price_df=US_VWAP,
               EAR_period=1,
               entry=2,
               long_criteria=((2,1000),(0.05,1000),None,None),
               short_criteria=((-1000,-2),(-1000,-0.03),None,(5000,10000000)),
               holding=30,
               start=pd.Timestamp(2010,1,1),
               end=pd.Timestamp(2019,3,29),
               old_position=False,
               new_signal=True,
               revision_adjust=(True,(0.05,0.1),(-0.1,-0.03),2),
               early_exit=False)

In [ ]:
_long,_short,_alpha,_portfolio=US_new.signal_account(stop=[20,18,"rel"],
                                                           gross=(30,15),
                                                           index_df=(0.5*abs_return_index_US.loc["SPX Index"]+\
                                                          0.5*abs_return_index_US.loc["RTY Index"]).to_frame("US_index"),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=65)

In [ ]:
plot_signal("US new signal - no stop",[10,4],US_new.portfolio_account)

In [ ]:
plot_signal("US new signal - abs stop 100k",[10,4],US_new.portfolio_account)

In [ ]:
plot_signal("US new signal - abs stop 200k",[10,4],US_new.portfolio_account)

In [ ]:
plot_signal("US new signal - rel stop 100k",[10,4],US_new.portfolio_account)

In [ ]:
plot_signal("US new signal - rel stop 200k",[10,4],US_new.portfolio_account)

In [ ]:
US_orig=signal(fundamental_df=US1_old,
                price_df=US_price,
                EAR_period=1,
                entry=2,
                long_criteria=((3,1000),(0.05,1000),None,None),
                short_criteria=((-1000,-2),(-1000,-0.03),None,(5000,10000000)),
                holding=30,
                start=pd.Timestamp(2010,1,1),
                end=pd.Timestamp(2019,3,29),
                old_position=False,
                new_signal=False,
                revision_adjust=(False,(0.05,0.2),(-0.1,-0.03),2),
               early_exit=False)

In [ ]:
_long,_short,_alpha,_portfolio=US_orig.signal_account(stop=None,
                                                           gross=(26,15),
                                                           index_df=(0.5*abs_return_index_US.loc["SPX Index"]+\
                                                          0.5*abs_return_index_US.loc["RTY Index"]).to_frame("US_index"),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=65)

In [ ]:
plot_signal("US orig signal - no exit",[10,4],US_orig.portfolio_account)

# 6 - Fitting choice

In [ ]:
def fitting_df(long,EAR_list,revision_list,fundamental_df,price_df,EAR_period,holding,start,end,stop,gross,index_df,
               net_level,risk_parity,liquidity,capital,exclusive,old_position,new_signal):
    '''
    Create a dataframe of sharpe and drawdown for different fitting variations
    EAR/revision list includes the upper bound
    Exclusive means gap and non eclusive means from that to the maximum
    '''
    year_list=range(start.year,end.year+1)
    EAR_list.sort()
    revision_list.sort()
    
    criteria_list_short=list(itertools.product(*[EAR_list[1:],revision_list[1:]]))
    criteria_list_long=list(itertools.product(*[EAR_list[:-1],revision_list[:-1]]))
    
    sharpe_dict={}
    dd_dict={}
    if exclusive:
        if long:
            for s in criteria_list_long:
                signal_class=signal(fundamental_df=fundamental_df,
                                   price_df=price_df,
                                   EAR_period=EAR_period,
                                   long_criteria=((s[0],EAR_list[EAR_list.index(s[0])+1]),\
                                                  (s[1],revision_list[revision_list.index(s[1])+1]),None,None),
                                   short_criteria=None,
                                   holding=holding,
                                   start=start,
                                   end=end,
                                   old_position=old_position,
                                   new_signal=new_signal)
                
                portfolio=signal_class.signal_account(stop=stop,
                                                      gross=gross,
                                                      index_df=index_df.copy(),
                                                      net_level=net_level,
                                                      risk_parity=risk_parity,
                                                      liquidity=liquidity,
                                                      capital=capital)

                sharpe_dict[s]=sharpe_by_year(portfolio[-1][1],year_list)
                dd_dict[s]=dd_by_year(portfolio[-1][1],year_list)
                
        else:
            for s in criteria_list_short:
                signal_class=signal(fundamental_df=fundamental_df,
                                   price_df=price_df,
                                   EAR_period=EAR_period,
                                   long_criteria=None,
                                   short_criteria=((EAR_list[EAR_list.index(s[0])-1],s[0]),\
                                                   (revision_list[revision_list.index(s[1])-1],s[1]),None,(5000,10000000)),
                                   holding=holding,
                                   start=start,
                                   end=end,
                                   old_position=old_position,
                                   new_signal=new_signal)

                portfolio=signal_class.signal_account(stop=stop,
                                                      gross=gross,
                                                      index_df=index_df.copy(),
                                                      net_level=net_level,
                                                      risk_parity=risk_parity,
                                                      liquidity=liquidity,
                                                      capital=capital)

                sharpe_dict[s]=sharpe_by_year(portfolio[-1][1],year_list)
                dd_dict[s]=dd_by_year(portfolio[-1][1],year_list)
    
    else:
        if long:
            for i in criteria_list_long:
                signal_class=signal(fundamental_df=fundamental_df,
                                   price_df=price_df,
                                   EAR_period=EAR_period,
                                   long_criteria=((i[0],10),(i[1],10),None,None),
                                   short_criteria=None,
                                   holding=holding,
                                   start=start,
                                   end=end,
                                   old_position=old_position,
                                   new_signal=new_signal)

                portfolio=signal_class.signal_account(stop=stop,
                                                      gross=gross,
                                                      index_df=index_df.copy(),
                                                      net_level=net_level,
                                                      risk_parity=risk_parity,
                                                      liquidity=liquidity,
                                                      capital=capital)

                sharpe_dict[i]=sharpe_by_year(portfolio[-1][1],year_list)
                dd_dict[i]=dd_by_year(portfolio[-1][1],year_list)
                
        else:
            for i in criteria_list_short:
                signal_class=signal(fundamental_df=fundamental_df,
                                   price_df=price_df,
                                   EAR_period=EAR_period,
                                   long_criteria=None,
                                   short_criteria=((-10,i[0]),(-10,i[1]),None,(5000,10000000)),
                                   holding=holding,
                                   start=start,
                                   end=end,
                                   old_position=old_position,
                                   new_signal=new_signal)
                
                portfolio=signal_class.signal_account(stop=stop,
                                                      gross=gross,
                                                      index_df=index_df.copy(),
                                                      net_level=net_level,
                                                      risk_parity=risk_parity,
                                                      liquidity=liquidity,
                                                      capital=capital)

                sharpe_dict[i]=sharpe_by_year(portfolio[-1][1],year_list)
                dd_dict[i]=dd_by_year(portfolio[-1][1],year_list)

    sharpe_df=pd.DataFrame(sharpe_dict)
    dd_df=pd.DataFrame(dd_dict)
        
    return sharpe_df,dd_df
        

In [ ]:
Europe_long_sharpe,Europe_long_dd=fitting_df(True,[1,2,3,4],[0.02,0.03,0.04,0.05,10],Europe2,Europe_price,2,30,\
                                             pd.Timestamp(2010,1,1),pd.Timestamp(2018,12,31),(8,None),(30,None)\
                                             ,abs_return_index_Europe.loc["SX5E Index"].to_frame('Europe'),0,True,0.2,50,True,
                                            False,False)

In [ ]:
Europe_long_sharpe

In [ ]:
Europe_short_sharpe,Europe_short_dd=fitting_df(False,[-1,-2,-3,-4],[-0.02,-0.03,-0.04,-0.05,-0.06],Europe,Europe_price,2,30,\
                                             pd.Timestamp(2007,1,1),pd.Timestamp(2018,12,31),(8,8),(30,30)\
                                             ,abs_return_index_Europe.loc["SX5E Index"].to_frame('Europe'),0,True,0.2,50)

In [ ]:
Europe_short_sharpe.to_csv("Europe_short_sharpe.csv")

In [ ]:
Europe_short_dd.to_csv("Europe_short_dd.csv")

In [ ]:
US_long_sharpe,US_long_dd=fitting_df(True,[1,2,3,4],[0.02,0.03,0.04,0.05,0.06,0.07],\
                                     US,US_price,2,30,pd.Timestamp(2007,1,1),pd.Timestamp(2018,12,31),(8,None),(30,None)\
                                             ,(0.5*abs_return_index_US.loc["SPX Index"]+0.5*abs_return_index_US.loc["RTY Index"])\
                                     .to_frame('US'),0,True,0.2,50)

In [ ]:
adj_US_long_sharpe,adj_US_long_dd=fitting_df(True,[1,2,3,4],[0.02,0.03,0.04,0.05,0.06,0.07],\
                                     adj_US,US_price,2,30,pd.Timestamp(2007,1,1),pd.Timestamp(2018,12,31),(8,None),(30,None)\
                                             ,(0.5*abs_return_index_US.loc["SPX Index"]+0.5*abs_return_index_US.loc["RTY Index"])\
                                     .to_frame('US'),0,True,0.2,50,False)

In [ ]:
US_short_sharpe,US_short_dd=fitting_df(False,[-1,-2,-3,-4],[-0.02,-0.03,-0.04,-0.05,-0.06,-0.07],\
                                     US,US_price,2,30,pd.Timestamp(2007,1,1),pd.Timestamp(2018,12,31),(8,8),(30,30)\
                                             ,(0.5*abs_return_index_US.loc["SPX Index"]+0.5*abs_return_index_US.loc["RTY Index"])\
                                     .to_frame('US'),0,True,0.2,50)

In [ ]:
adj_US_short_sharpe,adj_US_short_dd=fitting_df(False,[-1,-2,-3],[-0.02,-0.03,-0.04,-0.05],\
                                     adj_US,US_price,2,30,pd.Timestamp(2007,1,1),pd.Timestamp(2018,12,31),(8,8),(30,30)\
                                             ,(0.5*abs_return_index_US.loc["SPX Index"]+0.5*abs_return_index_US.loc["RTY Index"])\
                                     .to_frame('US'),0,True,0.2,50,False)

In [ ]:
US_short_sharpe.to_csv("US_short_sharpe.csv")

In [ ]:
US_short_dd.to_csv("US_short_dd.csv")

In [ ]:
Asia_long_sharpe,Asia_long_dd=fitting_df(True,[1,2,3,4],[0.02,0.03,0.04,0.05,0.06,0.07],\
                                     Asia,Asia_price,2,20,pd.Timestamp(2007,1,1),pd.Timestamp(2018,12,31),(8,None),(30,None)\
                                             ,abs_return_index_Asia.loc[["AS51 Index","HSI Index",
                                                                                                "TPX Index","KOSPI Index",
                                                                                                "MXSG Index","TAMSCI Index"]].T\
                                     ,0,True,0.2,50)

In [ ]:
Asia_long_sharpe.to_csv("Asian_long_sharpe.csv")

In [ ]:
Asia_long_dd.to_csv("Asian_long_dd.csv")

# 7 - Drawdown analysis

In [ ]:
def drawdown_days(acct_curve,timestamp):
    dd=acct_curve-acct_curve.cummax()
    previous_high=acct_curve.loc[:timestamp].max()
    high_timestamp=acct_curve[acct_curve==previous_high].index[0]
    days=acct_curve.index.tolist().index(timestamp)-acct_curve.index.tolist().index(high_timestamp)
    return (high_timestamp,days)

In [ ]:
def drawdown_df(portfolio_cache):
    drawdown=portfolio_cache[-1].iloc[:,-1].to_frame()
    drawdown.columns=["Index"]
    drawdown["Index vol"]=drawdown["Index"].rolling(30).std()
    drawdown["Portfolio"]=portfolio_cache[1].diff(1)
    drawdown["Account"]=portfolio_cache[1].copy()
    drawdown["Drawdown"]= drawdown["Account"]- drawdown["Account"].cummax()
    drawdown["Drawdown days"]= drawdown.apply(lambda x: drawdown_days(drawdown["Account"],x.name)[1],axis=1)
    drawdown["Drawdown start"]= drawdown.apply(lambda x: drawdown_days(drawdown["Account"],x.name)[0],axis=1)
    drawdown["Gross"]=np.abs(portfolio_cache[2]).sum(axis=1)
    return drawdown
    

In [ ]:
def portfolio_drawdown_df(portfolio_cache,stop,multiple):
    portfolio_cache_list=list(portfolio_cache)
    orig_size=portfolio_cache_list[2]
    orig_account=portfolio_cache_list[1]
    dd=orig_account-orig_account.cummax()
    stop_evaluate=dd[dd.shift(1)<stop]
    adj_size=orig_size.apply(lambda x: x*multiple if x.name in stop_evaluate.index else x,axis=1)
    
    new_portfolio_cache=portfolio_cache_list.copy()
    new_portfolio_cache[2]=adj_size
    new_portfolio_cache[0]=adj_size.shift(1)*new_portfolio_cache[-1]
    new_portfolio_cache[1]=new_portfolio_cache[0].sum(axis=1).cumsum()
    return tuple(new_portfolio_cache)
    

# 8- Quarterly review

In [ ]:
def review_df(signal_class,start,end,old_position):
    '''
    dataframe for review purpose
    '''
    try:
        long_abs_return=slice_universe(signal_class.long_df.copy(),start,end,old_position)
        long_abs_return[long_abs_return==0]=None
        long_rel_return=long_abs_return.subtract(signal_class.portfolio_account[-1].iloc[:,-1],axis=0)
        long_df=((1+long_rel_return).cumprod()-1).ffill().iloc[-1].to_frame()
        long_df["Ticker"]=long_df.apply(lambda x:x.name[0],axis=1)
        long_df["Date"]=long_df.apply(lambda x:datetime.strptime(x.name[1],"%d/%b/%Y"),axis=1)
        
        long_size=slice_universe(signal_class.long_cache[2].copy(),start,end,old_position)
        long_df["Size"]=long_size.apply(lambda x: x.dropna().iloc[0],axis=0)
    except:
        long_df=None

    try:
        short_abs_return=-slice_universe(signal_class.short_df.copy(),start,end,old_position)
        short_abs_return[short_abs_return==0]=None
        short_rel_return=short_abs_return.add(signal_class.portfolio_account[-1].iloc[:,-1],axis=0)
        short_df=((1+short_rel_return).cumprod()-1).ffill().iloc[-1].to_frame()
        short_df["Ticker"]=short_df.apply(lambda x:x.name[0],axis=1)
        short_df["Date"]=short_df.apply(lambda x:datetime.strptime(x.name[1],"%d/%b/%Y"),axis=1)
        
        short_size=slice_universe(signal_class.short_cache[2].copy(),start,end,old_position)
        short_df["Size"]=short_size.apply(lambda x: x.dropna().iloc[0],axis=0)
    except:
        short_df=None
        
    return long_df,short_df

In [ ]:
Asia_long,Asia_short=review_df(Asia_fit1,pd.Timestamp(2019,1,1),pd.Timestamp(2019,3,29),False)

In [ ]:
Asia_long.to_csv("Asia_long.csv")

# 9 - Sizing for the live trading

## 9.2 -  US live sizing

In [ ]:
US_live_orig=signal(fundamental_df=US2,
                   price_df=US_price,
                   EAR_period=1,
               entry=2,
                   long_criteria=((3,1000),(0.05,1000),None,None),
                   short_criteria=((-1000,-2),(-1000,-0.03),None,(5000,10000000)),
                   holding=30,
                   start=pd.Timestamp(2019,1,1),
                   end=pd.Timestamp(2019,3,29),
                  old_position=False,
                  new_signal=False,
                 revision_adjust=(False,(0.05,0.2),(-0.1,-0.03),2))

In [ ]:
_long_US_live,_short_US_live,_alpha_US_live,_portfolio_US_live=US_live_orig.signal_account(stop=(8,8),
                                                           gross=(30,15),
                                                           index_df=(0.5*abs_return_index_US.loc["SPX Index"]+\
                                                          0.5*abs_return_index_US.loc["RTY Index"]).to_frame("US_index"),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=65)

In [ ]:
US_live.plot_account("US reference")

In [ ]:
'''
US reference sizing and volatility
'''
print("US long size is "+str(_long_US_live[2].apply(lambda x:x.dropna().iloc[0],axis=0).mean()))
print("US long volatility is "+str(US2.loc[_long_US_live[-1].index]["30d_vol"].mean()))
print("US long number is "+str(_long_US_live[2].count(axis=1).mean()))
print("US short size is "+str(_short_US_live[2].apply(lambda x:x.dropna().iloc[0],axis=0).mean()))
print("US short volatility is "+str(US2.loc[_short_US_live[-1].index]["30d_vol"].mean()))
print("US short number is "+str(_short_US_live[2].count(axis=1).mean()))

In [ ]:
US_live_new=signal(fundamental_df=US1,
                   price_df=US_VWAP,
                   EAR_period=1,
               entry=2,
                   long_criteria=((2,1000),(0.05,1000),None,None),
                   short_criteria=((-1000,-2),(-1000,-0.03),None,(5000,10000000)),
                   holding=30,
                   start=pd.Timestamp(2019,1,1),
                   end=pd.Timestamp(2019,3,29),
                  old_position=False,
                  new_signal=True,
                 revision_adjust=(True,(0.05,0.2),(-0.1,-0.03),2))

In [ ]:
_long_US,_short_US,_alpha_US,_portfolio_US=US_live_new.signal_account(stop=(8,8),
                                                           gross=(30,15),
                                                           index_df=(0.5*abs_return_index_US.loc["SPX Index"]+\
                                                          0.5*abs_return_index_US.loc["RTY Index"]).to_frame("US_index"),
                                                                                      net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=65)

In [ ]:
_,long_lower_size_US=sizing(US_live_new.long_df,US_live_new.long_df,30,US_live_new.fundamental_df,True,abs_return_US\
                    ,True,0.2,100, US_live_new.revision_adjust, True)

In [ ]:
_,short_lower_size_US=sizing(US_live_new.short_df,US_live_new.short_df,30,US_live_new.fundamental_df,True,abs_return_US\
                    ,True,0.2,100, US_live_new.revision_adjust,False)

In [ ]:
'''
US reference sizing and volatility
'''
print("US long size is "+str(long_lower_size_US))
print("US long volatility is "+str(US1.loc[_long_US[-1].index]["30d_vol"].mean()))
print("US long number is "+str(_long_US[2].count(axis=1).mean()))
print("US short size is "+str(short_lower_size_US))
print("US short volatility is "+str(US1.loc[_short_US[-1].index]["30d_vol"].mean()))
print("US short number is "+str(_short_US[2].count(axis=1).mean()))

## 9.2 - Europe live sizing

In [ ]:
Europe_live_old=signal(fundamental_df=Europe2,
                    price_df=Europe_price,
                   EAR_period=1,
                    entry=2,
                   long_criteria=((1,1000),(0.03,1000),None,None),
                   short_criteria=((-1000,-2),(-1000,-0.03),None,(5000,10000000)),
                   holding=30,
                   start=pd.Timestamp(2019,1,1),
                   end=pd.Timestamp(2019,3,29),
                  old_position=False,
                  new_signal=False,
                 revision_adjust=(False,0.02,0.1,2))

In [ ]:
_long_Europe_live,_short_Europe_live,_alpha_Europe_live,_portfolio_Europe_live=Europe_live_old.signal_account(stop=(8,8),
                                                           gross=(30,15),
                                                           index_df=abs_return_index_Europe.loc["SX5E Index"].to_frame('Europe'),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=65)

In [ ]:
Europe_live1.plot_account("Europe reference 1")

In [ ]:
'''
Europe reference sizing and volatility
'''
print("Europe long size is "+str(_long_Europe_live[2].apply(lambda x:x.dropna().iloc[0],axis=0).mean()))
print("Europe long volatility is "+str(Europe2.loc[_long_Europe_live[-1].index]["30d_vol"].mean()))
print("Europe long number is "+str(_long_Europe_live[2].count(axis=1).mean()))
print("Europe short size is "+str(_short_Europe_live[2].apply(lambda x:x.dropna().iloc[0],axis=0).mean()))
print("Europe short volatility is "+str(Europe2.loc[_short_Europe_live[-1].index]["30d_vol"].mean()))
print("Europe short number is "+str(_short_Europe_live[2].count(axis=1).mean()))

In [ ]:
Europe_live_new=signal(fundamental_df=Europe2,
                   price_df=Europe_VWAP,
                   EAR_period=1,
                   entry=3,
                   long_criteria=((2,1000),(0.02,1000),None,None),
                   short_criteria=((-1000,-2),(-1000,-0.02),None,(5000,10000000)),
                   holding=30,
                   start=pd.Timestamp(2019,1,1),
                   end=pd.Timestamp(2019,3,29),
                  old_position=False,
                  new_signal=True,
                 revision_adjust=(True,(0.02,0.05),(-0.05,-0.02),2))

In [ ]:
_long_Europe,_short_Europe,_alpha_Europe,_portfolio_Europe=Europe_live_new.signal_account(stop=(8,8),
                                                           gross=(30,15),
                                                           index_df=abs_return_index_Europe.loc["SX5E Index"].to_frame('Europe'),
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=65)

In [ ]:
_,long_lower_size=sizing(Europe_live_new.long_df,Europe_live_new.long_df,30,Europe_live_new.fundamental_df,True,abs_return_Europe\
                    ,True,0.2,100, Europe_live_new.revision_adjust, True)

In [ ]:
_,short_lower_size=sizing(Europe_live_new.short_df,Europe_live_new.short_df,30,Europe_live_new.fundamental_df,True,abs_return_Europe\
                    ,True,0.2,100, Europe_live_new.revision_adjust,False)

In [ ]:
'''
Europe reference sizing and volatility
'''
print("Europe long size is "+str(long_lower_size))
print("Europe long volatility is "+str(Europe2.loc[_long_Europe[-1].index]["30d_vol"].mean()))
print("Europe long number is "+str(_long_Europe[2].count(axis=1).mean()))
print("Europe short size is "+str(short_lower_size))
print("Europe short volatility is "+str(Europe2.loc[_short_Europe[-1].index]["30d_vol"].mean()))
print("Europe short number is "+str(_short_Europe[2].count(axis=1).mean()))

In [ ]:
Asia_live_new=signal(fundamental_df=Asia2,
                   price_df=Asia_VWAP,
                   EAR_period=1,
                entry=2,
                   long_criteria=((1,1000),(0.02,1000),None,None),
                   short_criteria=None,
                   holding=20,
                   start=pd.Timestamp(2010,1,1),
                   end=pd.Timestamp(2019,3,29),
                  old_position=False,
                  new_signal=True,
                 revision_adjust=(True,(0.02,0.05),(-0.1,-0.02),2))

In [ ]:
Asia_live_old=signal(fundamental_df=Asia2,
                   price_df=Asia_price,
                   EAR_period=1,
                entry=2,
                   long_criteria=((1,1000),(0.02,1000),None,None),
                   short_criteria=None,
                   holding=20,
                   start=pd.Timestamp(2019,1,1),
                   end=pd.Timestamp(2019,3,29),
                  old_position=False,
                  new_signal=False,
                 revision_adjust=(False,(0.02,0.05),(-0.1,-0.02),2))

In [ ]:
_long_Asia_live,_short_Asia_live,_alpha_Asia_live,_portfolio_Asia_live=Asia_live_new.signal_account(stop=(8,None),
                                                           gross=(20,None),
                                                           index_df=abs_return_index_Asia.loc[["AS51 Index","HSI Index",
                                                                                                "TPX Index","KOSPI Index",
                                                                                                "MXSG Index","TAMSCI Index"]].T,
                                                           net_level=0,
                                                           risk_parity=True,
                                                           liquidity=0.2,
                                                           capital=17)

In [ ]:
'''
Europe reference sizing and volatility
'''
print("Asia long size is "+str(_long_Asia_live[2].apply(lambda x:x.dropna().iloc[0],axis=0).mean()))
print("Asia long volatility is "+str(Asia2.loc[_long_Asia_live[-1].index]["30d_vol"].mean()))
print("Asia long number is "+str(_long_Asia_live[2].count(axis=1).mean()))


